# Interacciones entre partículas cercanas

In [ ]:
import cv2
import numpy as np
from IPython import display as display

import ipywidgets as ipw
import PIL
from io import BytesIO

# Definición de Sistema

In [ ]:
NUM_PARTICULAS = 20
MAX_X = 450
MAX_Y = 450
X0 = 50
Y0 = 50

class System:
    def __init__(self):
        self.x = np.linspace(X0, MAX_X, NUM_PARTICULAS)
        self.y = np.linspace(Y0, MAX_Y, NUM_PARTICULAS)
        self.r = np.ones((NUM_PARTICULAS, NUM_PARTICULAS))
        self.canIncrease = np.ones((NUM_PARTICULAS, NUM_PARTICULAS))
        self.img = np.zeros((500, 500, 3), dtype="uint8")

        self.wIm = ipw.Image()
        display.display(self.wIm)

        self.r[np.random.randint(NUM_PARTICULAS), np.random.randint(NUM_PARTICULAS)] = 3
        self.frame = 0
        self.graficar()
    
    def actualizar_particulas(self):
        for i in range(NUM_PARTICULAS-1):
            for k in range(NUM_PARTICULAS-1):
                if 0<i<NUM_PARTICULAS and 0<k<NUM_PARTICULAS and self.canIncrease[i, k]==1:
                    self.r[i, k] += 0.05*self.r[i-1, k] + 0.05*self.r[i+1, k]
                    self.r[i, k] += 0.05*self.r[i, k-1] + 0.05*self.r[i, k+1]
                
                if self.canIncrease[i, k]==1 and self.r[i, k] >= 8:
                    self.canIncrease[i, k] = 0
                if self.canIncrease[i, k]==0 and self.r[i, k] <= 3:
                    self.canIncrease[i, k] = 1
                
                #decay
                self.r[i, k] *= 0.9


    def graficar(self):
        r=1
        color = (255,255,255)

        for i in range(len(self.x)):
            x = self.x[i]
            for k in range(len(self.y)):
                y = self.y[k]
                cv2.circle(self.img, (int(x), int(y)), int(r+self.r[i, k]), color, -1)


    def iterar(self):
        self.frame += 1
        self.actualizar_particulas()
        self.img[:] = (0,0,0)
        cv2.putText(self.img, str(self.frame), (20,20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255))

        self.graficar()
        pilIm = PIL.Image.fromarray(self.img, mode="RGB")
        with BytesIO() as fOut:
            pilIm.save(fOut, format="png")
            byPng = fOut.getvalue()
                
        # set the png bytes as the image value; 
        # this updates the image in the browser.
        self.wIm.value=byPng
            

In [ ]:
s = System()

for _ in range(300):
    s.actualizar_particulas()
    s.iterar()

Image(value=b'')